In [0]:
import os
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import sklearn
import numpy
import scipy
import plotly
import bs4 as bs
import urllib.request
import boto3

In [0]:
%pip install torch==2.1.0

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
file_path = 's3a://my-bigdata-project-sa/landing/amazon_reviews_us_Beauty_v1_00.tsv'
sdf = spark.read.csv(file_path, sep='\t', header=True, inferSchema=True)

In [0]:
sdf.select(sdf.customer_id, sdf.review_body).show()

+-----------+---------------------+
|customer_id|          review_body|
+-----------+---------------------+
|    1797882| Love this, excell...|
|   18381298| The great thing a...|
|   19242472| Great Product, I'...|
|   19551372| I use them as sho...|
|   14802407| This is my go-to ...|
|    2909389|                 Good|
|   19397215| Great eyeliner, d...|
|    3195210|Best makeup remover！|
|   52216383| This is a great p...|
|   10278216| I thought it woul...|
|   24655453| These are great f...|
|   30788223| Just got this.  N...|
|   11257536| Wish I had discov...|
|   29605778| I bought this to ...|
|   41238422| The tint is aweso...|
|   23620123| I'm in love with ...|
|   25564245| I was given this ...|
|     801375| Magical everybody...|
|    9478730| Works fantastic! ...|
|   18418407| excellent battery...|
+-----------+---------------------+
only showing top 20 rows



In [0]:
sdf.filter(sdf.star_rating > 3 ).show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|          review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+---------------------+-----------+
|         US|    1797882|R3I2DHQBR577SS|B001ANOOOE|       2102612|The Naked Bee Vit...|          Beauty|          5|            0|          0|   N|                Y|          Five Stars| Love this, excell...| 2015-08-31|
|         US|   18381298|R1QNE9NQFJC2Y4|B0016J22EQ|     106393691|Alba Botanica Sun...|          Beauty|          5|

In [0]:
sdf.groupBy('customer_id').count().show()
sdf.groupBy('customer_id').agg({'helpful_votes': 'mean'}).collect()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|    9015608|    3|
|    9767880|    2|
|   15958313|    1|
|   12879980|   20|
|   43920023|   31|
|     106535|    1|
|   20328662|    1|
|   12819130|    4|
|   50145309|    1|
|   15159322|    2|
|   16747249|    2|
|   37678162|    1|
|     227763|    1|
|   15191144|    2|
|   31946913|    1|
|   44295916|    1|
|   19021463|    1|
|     806337|    2|
|   16306618|   18|
|     393186|    1|
+-----------+-----+
only showing top 20 rows

Out[5]: [Row(customer_id=9015608, avg(helpful_votes)=3.0),
 Row(customer_id=9767880, avg(helpful_votes)=0.0),
 Row(customer_id=15958313, avg(helpful_votes)=0.0),
 Row(customer_id=12879980, avg(helpful_votes)=4.45),
 Row(customer_id=43920023, avg(helpful_votes)=0.7),
 Row(customer_id=106535, avg(helpful_votes)=0.0),
 Row(customer_id=20328662, avg(helpful_votes)=0.0),
 Row(customer_id=12819130, avg(helpful_votes)=0.0),
 Row(customer_id=50145309, avg(helpful_votes)=0.0),
 Row(customer_id=1515

In [0]:
sdf.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = false)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)



In [0]:
sdf.count()

Out[18]: 5115666

In [0]:
sdf = sdf.sample(False, 0.25)

In [0]:
sdf.summary().show()

+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+-----------------+------------------+------------------+-------+-----------------+--------------------+--------------------+
|summary|marketplace|         customer_id|     review_id|product_id|     product_parent|       product_title|product_category|      star_rating|     helpful_votes|       total_votes|   vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+--------------------+--------------+----------+-------------------+--------------------+----------------+-----------------+------------------+------------------+-------+-----------------+--------------------+--------------------+
|  count|    1277358|             1277358|       1277358|   1277358|            1277358|             1277358|         1277312|          1277312|           1277312|           1277312|1277312|          1277312|             1277311|             12771

In [0]:
sdf.select("star_rating", "helpful_votes", "total_votes").summary("count", "min", "max", "mean").show()

+-------+-----------------+------------------+------------------+
|summary|      star_rating|     helpful_votes|       total_votes|
+-------+-----------------+------------------+------------------+
|  count|          1277312|           1277312|           1277312|
|    min|                1|                 0|                 0|
|    max|                5|              5789|              5895|
|   mean|4.186295126014631|1.7166886398937768|2.1703303499849684|
+-------+-----------------+------------------+------------------+



In [0]:
sdf.select("review_headline", "review_body").summary("count", "min", "max").show()

+-------+--------------------+--------------------+
|summary|     review_headline|         review_body|
+-------+--------------------+--------------------+
|  count|             1277311|             1277134|
|    min|              |I loved this pro...|
|    max|🚫 A complete was...|🚨🚨🚨🚨This not ...|
+-------+--------------------+--------------------+



In [0]:
from pyspark.sql.functions import col, count, when, isnull

sdf.select([count(when(~isnull(col(c)), c)).alias(c) for c in ["star_rating", "review_body"]]).show()

+-----------+-----------+
|star_rating|review_body|
+-----------+-----------+
|    1277134|    1277134|
+-----------+-----------+



In [0]:
sdf = sdf.na.drop(subset=["star_rating", "review_body"])

In [0]:
def ascii_only(mystring):
    if mystring:
        return mystring.encode('ascii', 'ignore').decode('ascii')
    else:
        return None

ascii_udf = udf(ascii_only)
sdf = sdf.withColumn("clean_review_headline", ascii_udf('review_headline'))
sdf = sdf.withColumn("clean_review_body", ascii_udf('review_body'))
sdf.select("clean_review_headline", "clean_review_body").summary("count", "min", "max").show()

+-------+---------------------+--------------------+
|summary|clean_review_headline|   clean_review_body|
+-------+---------------------+--------------------+
|  count|              1277133|             1277134|
|    min|                     |                    |
|    max| ~~~~ GET STOKED~~...|~~~~~~~~~~~~~~~~~...|
+-------+---------------------+--------------------+



In [0]:
sdf.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = false)
 |-- product_parent: integer (nullable = true)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- vine: string (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- clean_review_headline: string (nullable = true)
 |-- clean_review_body: string (nullable = true)



In [0]:
sdf = sdf.drop("product_parent", "vine")

In [0]:
sdf.printSchema()

root
 |-- marketplace: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- product_id: string (nullable = false)
 |-- product_title: string (nullable = true)
 |-- product_category: string (nullable = true)
 |-- star_rating: string (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)
 |-- verified_purchase: string (nullable = true)
 |-- review_headline: string (nullable = true)
 |-- review_body: string (nullable = true)
 |-- review_date: date (nullable = true)
 |-- clean_review_headline: string (nullable = true)
 |-- clean_review_body: string (nullable = true)



In [0]:
output_file_path="s3://my-bigdata-project-sa/raw/cleaned_amazon_reviews_us_Beauty_v1_00.parquet"
sdf.write.parquet(output_file_path)

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1928608511546478>:2
      1 output_file_path="s3://my-bigdata-project-sa/raw/cleaned_amazon_reviews_us_Beauty_v1_00.parquet"
----> 2 sdf.write.parquet(output_file_path)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1655, in DataFrameWriter.parquet(self, path, mode, partitionBy, compression)
   1653     self.partitionBy(partitionBy)
   1654 self._set_opts(compression=compression)
-> 1655 self._jwrite.parquet(path)

File /databricks/spark/python/li

In [0]:
df = spark.read.parquet('s3://my-bigdata-project-sa/raw/cleaned_amazon_reviews_us_Beauty_v1_00.parquet')
print(df.columns)

In [0]:
 # Define a function to strip out any non-ascii characters
def ascii_only(mystring):
  if mystring:
    return mystring.encode('ascii', 'ignore').decode('ascii')
  else:
    return None

In [0]:
# Turn this function into a User-Defined Function (UDF)
ascii_udf = udf(ascii_only)

In [0]:
# Clean the Reivew Body
sdf = sdf.withColumn("clean_review_body", ascii_udf('review_body'))

In [0]:
# Create a count of the review words
sdf = sdf.withColumn('review_body_wordcount', size(split(col('review_body'), ' ')))

In [0]:
# Filter out short review body texts
sdf = sdf.where(length(sdf.clean_review_body) > 10)

In [0]:
# Filter out review body texts with 5 or fewer words
sdf = sdf.where(sdf.review_body_wordcount > 5)

In [0]:
# Set up the tokenizer TF and IDF
tokenizer = RegexTokenizer(inputCol="clean_review_body", outputCol="clean_review_words", pattern="\\w+", gaps=False)
sdf = tokenizer.transform(sdf)

In [0]:
# Run the hash function over the tokens
hashtf = HashingTF(numFeatures=2**16, inputCol="clean_review_words", outputCol='clean_review_tf')
sdf = hashtf.transform(sdf)

In [0]:
# Create the inverse document frequency vectors (this can take a long time to complete)
idf = IDF(inputCol='clean_review_tf', outputCol="clean_review_features", minDocFreq=5)
sdf = idf.fit(sdf).transform(sdf)